In [1]:
!pip install onnx
!pip install tf2onnx
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.4 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import layers

import tf2onnx
import onnx
import time
import numpy as np
import onnxruntime as ort

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [4]:
x_train.shape, y_train.shape

((50000, 32, 32, 3), (50000, 1))

In [5]:
x_train = x_train / 255.0
x_test = x_test /255.0

In [7]:
model = keras.Sequential()
model.add(layers.Conv2D(128, kernel_size = (2,2), input_shape=(32,32,3) ))
model.add(keras.layers.Activation('relu'))
model.add(layers.MaxPool2D())
model.add(layers.Dropout(0.2))
model.add(layers.Conv2D(64, kernel_size = (2,2)))
model.add(keras.layers.Activation('relu'))
model.add(layers.MaxPool2D())
model.add(layers.Dropout(0.2))
model.add(layers.Conv2D(32, kernel_size = (2,2)))
model.add(keras.layers.Activation('relu'))
model.add(layers.MaxPool2D())
model.add(layers.Dropout(0.2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(32))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Dense(16))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Dense(10))
model.add(keras.layers.Activation('softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
model.compile(optimizer = tf.optimizers.Adam(), loss=tf.losses.sparse_categorical_crossentropy, metrics = ['accuracy'] )

In [9]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 31, 31, 128)         │           1,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_6 (Activation)            │ (None, 31, 31, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 15, 15, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 15, 15, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 14, 14, 64)          │          32,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_7 (Activation)            │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 7, 7, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 7, 7, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 6, 6, 32)            │           8,224 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_8 (Activation)            │ (None, 6, 6, 32)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 3, 3, 32)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 3, 3, 32)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 288)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 32)                  │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_9 (Activation)            │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_10 (Activation)           │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 10)                  │             170 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_11 (Activation)           │ (None, 10)                  │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 52,666 (205.73 KB)

 Trainable params: 52,666 (205.73 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
hist = model.fit(x_train, y_train, epochs=40, batch_size= 32, validation_data=(x_test, y_test))

Epoch 1/40
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - accuracy: 0.2761 - loss: 1.9287 - val_accuracy: 0.4612 - val_loss: 1.4775
Epoch 2/40
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.4759 - loss: 1.4476 - val_accuracy: 0.5184 - val_loss: 1.3504
Epoch 3/40
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.5261 - loss: 1.3140 - val_accuracy: 0.5743 - val_loss: 1.2124
Epoch 4/40
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.5553 - loss: 1.2326 - val_accuracy: 0.6130 - val_loss: 1.0946
Epoch 5/40
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.5892 - loss: 1.1549 - val_accuracy: 0.6394 - val_loss: 1.0397
Epoch 6/40
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.6055 - loss: 1.1098 - val_accuracy: 0.6363 - val_loss: 1.0246
Epoch 7/40
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.6138 - loss: 1.0776 - val_accuracy: 0.6537 - val_loss: 0.9891
Epoch 8/40
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.6318 - loss: 1.037

In [11]:
# save .tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('cfar10_model_tflite.tflite', 'wb') as f:
  f.write(tflite_model)

Saved artifact at '/tmp/tmp36odz9zh'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name='keras_tensor_92')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  137686456196208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137686456199200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137684587209200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137684587203744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137686455944816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137686476321744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137684584961952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137684584964768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137684584969168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137684584966000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137684585239

In [12]:
#save .h5 model
model.save('cfar10_h5.h5')

In [13]:
# save .onnx model
model.output_names = ['output']


spec =(tf.TensorSpec((None, 32, 32, 3), tf.float32, name='input'), )
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature=spec)
onnx.save_model(onnx_model, 'cfar10_saved_onnx.onnx')

In [14]:
# get model size function
import os
def model_size(name_file, file_path):
  model_size= os.path.getsize(file_path)
  model_size = model_size /(1024*1024)
  print(f'{name_file} size is :{model_size:.2f} Mb')


In [15]:
sample = x_test[0].reshape(-1, 32,32, 3)
sample.shape

(1, 32, 32, 3)

### The size of each model in MB

In [16]:
model_size('h5_model', '/content/cfar10_h5.h5')
model_size('tilite_model', '/content/cfar10_model_tflite.tflite')
model_size('onnx_model', '/content/cfar10_saved_onnx.onnx')


h5_model size is :0.67 Mb
tilite_model size is :0.21 Mb
onnx_model size is :0.20 Mb


### Calculate inference time for each model

In [25]:
# load and predict .h5 model for one sample
start = time.time()
h5_model = keras.models.load_model('/content/cfar10_h5.h5')
predict = h5_model.predict(sample)
predict_class =np.argmax(predict)
print(y_test[0])
print(predict_class)
finish = time.time()
h5_time= finish - start


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 304ms/step
[3]
3


In [18]:
# load and predict .tflite model for one sample

model_path = '/content/cfar10_model_tflite.tflite'
start = time.time()
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

input_detail = interpreter.get_input_details()
output_detail = interpreter.get_output_details()

interpreter.set_tensor(input_detail[0]['index'], sample.astype('float32'))
interpreter.invoke()
predict = interpreter.get_tensor(output_detail[0]['index'])
print(y_test[0])
print(np.argmax(predict))
finish = time.time()
tflite_time = finish - start

[3]
3


In [19]:
# load and predict .onnx model for one sample


onnx_model_path = '/content/cfar10_saved_onnx.onnx'
start = time.time()
onnx_model = onnx.load(onnx_model_path)

#input_shape = onnx_model.graph.input[0].type.tensor_type.shape.dim
#input_shape

session = ort.InferenceSession(onnx_model_path)

input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

prediction = session.run([output_name], {input_name: sample.astype('float32')})
print(y_test[0])
print(np.argmax(prediction[0]))
finish= time.time()
onnx_time =finish- start


[3]
3


In [20]:
print(h5_time)
print(tflite_time)
print(onnx_time)

0.7327415943145752
0.04320096969604492
0.00800943374633789


#### Evalute accurcay of each model

In [21]:
#h5 accuracy

def h5_predict(model_path):
  model = keras.models.load_model(model_path)
  y_pred = model.predict(x_test)

  return np.argmax(y_pred, axis =-1)

h5_pred = h5_predict('/content/cfar10_h5.h5')
h5_accuracy= np.mean(np.expand_dims(h5_pred,-1) == y_test)

h5_accuracy



313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step


0.7215

In [22]:
#tflite accuracy

def tflite_predict(model_path):
  interpreter = tf.lite.Interpreter(model_path=model_path)
  interpreter.allocate_tensors()

  input_detail = interpreter.get_input_details()
  output_detail = interpreter.get_output_details()
  y_pred = []
  for i in range(len(x_test)):

    interpreter.set_tensor(input_detail[0]['index'], x_test[i:i+1].astype('float32'))
    interpreter.invoke()
    predict = interpreter.get_tensor(output_detail[0]['index'])
    y_pred.append(np.argmax(predict, axis=-1 ))

  return(np.array(y_pred))

tflite_pred = tflite_predict('/content/cfar10_model_tflite.tflite')
tflite_accuracy= np.mean(tflite_pred == y_test)
tflite_accuracy

0.7215

In [23]:
#onnx accuracy

def onnx_predict(model_path):
  onnx_model = onnx.load(model_path)

  #input_shape = onnx_model.graph.input[0].type.tensor_type.shape.dim
  #input_shape

  session = ort.InferenceSession(model_path)

  input_name = session.get_inputs()[0].name
  output_name = session.get_outputs()[0].name
  y_pred = []
  for i in range(len(x_test)):

    prediction = session.run([output_name], {input_name: x_test[i:i+1].astype('float32')})
    y_pred.append(np.argmax(prediction[0],axis = 1))


  return(np.array(y_pred))


onnx_pred = onnx_predict('/content/cfar10_saved_onnx.onnx')
onnx_accuracy= np.mean(onnx_pred == y_test)
onnx_accuracy


0.7215

finish!